In [17]:
#load the data

import csv
import pandas as pd
import numpy as np
import scipy as sp 

with open('table.dat', 'r') as file:
    table = pd.read_csv(file, delimiter=',', header = None)

with open('msa.dat', 'r') as file:
    msa = pd.read_csv(file, delimiter=',', header = None)

with open('branchlength.dat', 'r') as file:
    brlen = pd.read_csv(file, delimiter=',', header = None)

msa[[0,1]] = msa[0].str.split(' ', expand=True)
msa[0] = pd.to_numeric(msa[0])
print(table)
print(msa)
print(brlen)

   0  1
0  9  1
1  9  2
2  8  9
3  8  3
4  7  4
5  7  5
6  6  7
7  6  8
   0                               1
0  1  AGATCAAGATCAAGATCAAGATCAAGATCA
1  2  AGCTCAAGCTCAAGCTCAAGCTCAAGCTCA
2  3  CGCTATCGCTATCGCTATCGCTATCGCTAT
3  4  CGTTACCGTTACCGTTACCGTTACCGTTAC
4  5  CGCTACCGCTACCGCTACCGCTACCGCTAC
     0    1     2     3    4     5     6     7
0  0.1  0.4  0.01  0.04  0.2  0.08  0.12  0.14


In [9]:
#define the class

class Node:
    def __init__(self, number, parent, sibling, seq, lenght):
        self.number = number
        self.parent = parent
        self.sibling = sibling
        self.seq = seq
        self.length = lenght

class Tree:
    def __init__(self, position):
        self.position = position

        

In [10]:
def create_node_list(tab, ms, brl):
    node_list = []
    nodes = {}
    branches = iter(brl.iloc[0])
    for i in range(len(tab[1])):
            
        itself = tab[1][i]
        parent =  tab[0][i]
        sib_index = [index for index, value in enumerate(tab[0]) if value == parent and tab[1][index] != itself]
        sibling = tab[1][sib_index[0]]
        branch = next(branches)
        sequence = np.array(list(ms[1][itself])) if itself in ms[0] else None       
        nodes[itself] = Node(itself, parent, sibling, sequence, branch)
        node_list.append(nodes[itself])
    for i in tab[0]:

        if i not in list(tab[1]):
            itself = i
            parent =  None
            sibling = None
            branch = None
            sequence = np.array(list(ms[1][itself])) if itself in ms[0] else None
            nodes[itself] = Node(itself, parent, sibling, sequence, branch)
            node_list.append(nodes[itself])
            break
    return node_list


In [11]:
tree_test = Tree(create_node_list(table, msa, brlen))
print(len(tree_test.position))

9


array([[0.03, 0.04, 0.35],
       [0.04, 0.12, 0.14]])

In [26]:
# Need to create function within the tree class to calculate the likelihood of the tree
#Create the likelyhood matrix of known sequences

base_prob = {"A": np.array([1,0,0,0]), "T":np.array([0,1,0,0]), "C" : np.array([0,0,1,0]), "G" : np.array([0,0,0,1])}

map_values = np.vectorize(lambda x: base_prob[x], signature='()->(n)')

seq_prob1 = map_values(tree_test.position[0].seq)
seq_prob2 = map_values(tree_test.position[1].seq)
Q = np.array([[-0.5625,0.1875,0.1875,0.1875],[0.1875,-0.5625,0.1875,0.1875],
 [0.1875,0.1875,-0.5625,0.1875],[0.1875,0.1875,0.1875,-0.5625]])

P1 = sp.linalg.expm(tree_test.position[0].length * Q ) 
P2 = sp.linalg.expm(tree_test.position[1].length * Q )
vec1 = np.matmul(seq_prob1, P1)
vec2 = np.matmul(seq_prob2, P2)
vec_root= vec1 * vec2